# Data assimilation for FEM solutions on the dyadic-subdivided unit square

In [20]:
import numpy as np
import importlib
import dyadic_fem as df
import matplotlib.pyplot as plt
importlib.reload(df)
%matplotlib inline

In [25]:
importlib.reload(df)

field_div = 2
fem_div = 6
coarse_div = 3

# First we generate our solution
a = df.DyadicRandomField(div=field_div, a_bar=1.0, c=0.99, seed=3)
fem = df.DyadicFEMSolver(div=fem_div, rand_field=a, f=1.0)
fem.solve()

coarse_grid = 2**coarse_div - 1
# Now make our bases
hat_b_L2 = df.make_hat_basis(coarse_div, space='L2')
hat_b_H1 = df.make_hat_basis(coarse_div, space='H1')
sin_b_L2 = df.make_sine_basis(coarse_div, N=coarse_grid, M=coarse_grid, space='L2')
sin_b_H1 = df.make_sine_basis(coarse_div, N=coarse_grid, M=coarse_grid, space='H1')




In [30]:
isinstance(a.field, df.DyadicPWConstant)

True